adapted from: https://www.kaggle.com/code/kanncaa1/recurrent-neural-network-with-pytorch

In [386]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from tweedejaars_project import load_df

In [387]:
""" Prepare Dataset """

# load data
df = load_df()
highly_correlated_features = ['naive_strategy_action', 'min_ptu_price_known',
                              'min_price_published', 'forecast_wind',
                              'time_since_last_two_sided',
                              'two_sided_daily_count', 'mid_price_published',
                              'vwap_avg', 'vwap_std', 'vwap_qty_sum',
                              'minute_in_ptu', 'downward_dispatch_published',
                              'settlement_price_bestguess', 'import_capacity',
                              'upward_dispatch_published',
                              'settlement_price_realized', 'forecast_solar']

df = df.dropna()

# split data into features and target
targets_numpy = df['target_two_sided_ptu']
features_numpy = df[highly_correlated_features]

# train test split
# could be replaced with scikit-learn TimeSeriesSplit
X_train, X_test = np.split(features_numpy, [int(.7 * len(features_numpy))])
y_train, y_test = np.split(targets_numpy, [int(.7 * len(targets_numpy))])

# create feature and targets tensor for train and test set.
featuresTrain = torch.Tensor(X_train.to_numpy(dtype=np.float64))
targetsTrain = torch.Tensor(y_train.to_numpy(dtype=np.float64))

featuresTest = torch.Tensor(X_test.to_numpy(dtype=np.float64))
targetsTest = torch.Tensor(y_test.to_numpy(dtype=np.float64))

# force the right dimension
targetsTrain = targetsTrain.unsqueeze(1)
targetsTest = targetsTest.unsqueeze(1)

# Pytorch train and test sets
train = torch.cat((featuresTrain, targetsTrain), dim=1)
test = torch.cat((featuresTest, targetsTest), dim=1)

# batch_size, epoch and iteration
first_batch_size = 15
n_iters = 100000
num_epochs = int(n_iters / (len(X_train) / first_batch_size))

# data loader makes batches and iterable
train_loader = DataLoader(train, batch_size = first_batch_size, shuffle=False)
test_loader = DataLoader(test, batch_size = first_batch_size, shuffle=False)

/home/tycho/miniconda3/envs/tweedejaars_project/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/home/tycho/miniconda3/envs/tweedejaars_project/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


In [417]:
""" Create RNN Model """

class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(RNNModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.output_dim = output_dim
        
        # RNN
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers, batch_first=True,
                          nonlinearity='relu')
        
        # Collection layer
        self.fc = nn.Linear(hidden_dim, output_dim)

        # Range redistribution [0,1]
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):

        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.shape[0], self.hidden_dim,
                         requires_grad=True)
        
        try:
            h0 = hn
        except:
            pass

        # One time step
        rnn_out, hn = self.rnn(x, h0)
        
        fc_out = self.fc(rnn_out[:, -1])  # last output

        output = self.sigmoid(fc_out)

        return output


# Create RNN
input_dim = 17    # input dimension
hidden_dim = 60   # hidden layer dimension
num_layers = 2    # number of hidden layers
output_dim = 15   # output dimension
seq_length = 15   # the number of time steps in each sequence

model = RNNModel(input_dim, hidden_dim, num_layers, output_dim)

# SGD Optimizer
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [419]:
""" Run model """

batch_size = int(first_batch_size / seq_length) 
loss_list = []
iteration_list = []
accuracy_list = []
count = 0
error = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    for batch in train_loader:
        
        # print(batch.shape)
        if batch.shape[0] != 15:
            break

        train = batch[:, :-1].reshape((batch_size, seq_length, input_dim))
        targets = batch[:, -1:]

        # Clear gradients
        optimizer.zero_grad()
        
        # Forward propagation
        # print(batch_size, seq_length, input_dim)
        # print(train.shape)
        # print(batch[:1, :])
        outputs = model(train)
        
        # Calculate softmax and cross entropy loss
        # print("outputs\n",outputs)
        # print(outputs.T.shape, targets.shape)
        loss = error(outputs.T, targets)
        
        # Calculating gradients
        loss.backward()
        
        # Update parameters
        optimizer.step()
        
        count += 1
        
        # Calculate Accuracy
        if count % 5 == 0:
            correct = 0
            total = 0
            # Iterate through test dataset
            for batch in test_loader:
                
                if batch.shape[0] == 15:
                    features = batch[:, :-1].reshape((batch_size, seq_length, input_dim))
                    labels = batch[:,-1:]
                    
                    # Forward propagation
                    outputs = model(features)
                    
                    # Get predictions from the maximum value
                    predicted = torch.max(outputs.data, 1)[1]
                    
                    # Total number of labels
                    total += labels.size(0)
                    
                    correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / float(total)
            
            # store loss and iteration
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)
            if count % 10 == 0:
                # Print Loss
                print(f'Iteration: {count}  Loss: {"loss.data[0]"}  Accuracy: {accuracy} %')

Iteration: 10  Loss: loss.data[0]  Accuracy: 52.77777862548828 %
Iteration: 20  Loss: loss.data[0]  Accuracy: 52.77777862548828 %
Iteration: 30  Loss: loss.data[0]  Accuracy: 52.77777862548828 %
Iteration: 40  Loss: loss.data[0]  Accuracy: 52.77777862548828 %
Iteration: 50  Loss: loss.data[0]  Accuracy: 52.77777862548828 %
Iteration: 60  Loss: loss.data[0]  Accuracy: 52.77777862548828 %
Iteration: 70  Loss: loss.data[0]  Accuracy: 52.77777862548828 %
Iteration: 80  Loss: loss.data[0]  Accuracy: 52.77777862548828 %
Iteration: 90  Loss: loss.data[0]  Accuracy: 52.77777862548828 %
Iteration: 100  Loss: loss.data[0]  Accuracy: 52.77777862548828 %
Iteration: 110  Loss: loss.data[0]  Accuracy: 52.77777862548828 %
Iteration: 120  Loss: loss.data[0]  Accuracy: 52.77777862548828 %
Iteration: 130  Loss: loss.data[0]  Accuracy: 52.77777862548828 %
Iteration: 140  Loss: loss.data[0]  Accuracy: 52.77777862548828 %
Iteration: 150  Loss: loss.data[0]  Accuracy: 52.77777862548828 %
Iteration: 160  Los

KeyboardInterrupt: 

In [ ]:
""" Model Evaluation """